In [1]:
import deepmatcher as dm
import py_entitymatching as em
import pandas as pd

In [4]:
A = em.read_csv_metadata(r"C:\hw-8-Data_integration\Project\Record linkage\idea_final_schema0.csv", key='name', low_memory = False)
B = em.read_csv_metadata(r"C:\hw-8-Data_integration\Project\Record linkage\idea_final_schema0.csv", key='name', low_memory = False)

In [9]:
# Load the labeled data into a dataframe.
G = em.read_csv_metadata(r"C:\hw-8-Data_integration\Project\Record linkage\DATASETS\dataset.csv", 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
print('Number of labeled pairs:', len(G))

Number of labeled pairs: 1300


In [13]:
G = G.drop(columns = ['ltable_city', 'rtable_city'], axis = 1)

In [14]:
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(G, r"C:\hw-8-Data_integration\Project\Record linkage\DATASETS\Split", 'train.csv', 'valid.csv', 'test.csv',
              [3, 1, 1])

In [15]:
# Load the training data files from the disk. Ignore the "left_id" and "right_id" 
# columns for data preprocessing.
# The 'use_magellan_convention' parameter asks deepmatcher to use Magellan's 
# naming convention for the left and right table column name prefixes 
# ("ltable_", and "rtable_"), and also to consider "_id" as the ID column.
train, validation, test = dm.data.process(
    path=r'C:\hw-8-Data_integration\Project\Record linkage\DATASETS\Split',
    cache='train_cache.pth',
    train='train.csv',
    validation='valid.csv',
    test='test.csv',
    use_magellan_convention=True,
    ignore_columns=('ltable_id', 'rtable_id'))


Reading and processing data from "C:\hw-8-Data_integration\Project\Record linkage\DATASETS\Split\train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "C:\hw-8-Data_integration\Project\Record linkage\DATASETS\Split\valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "C:\hw-8-Data_integration\Project\Record linkage\DATASETS\Split\test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [16]:
# Create a hybrid model.
model = dm.MatchingModel(attr_summarizer='hybrid')

In [17]:
# Train the hybrid model with 10 training epochs, batch size of 16, positive-to-negative 
# ratio to be 3. We save the best model (with the 
# highest F1 score on the validation set) to 'hybrid_model.pth'.
model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=16,
    best_save_path='hybrid_model2.pth',
    pos_neg_ratio=3)

c:\Users\rocci\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
c:\Users\rocci\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 22151812
===>  TRAIN Epoch 1


c:\Users\rocci\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
c:\Users\rocci\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:2741: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
0% [█████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:58


Finished Epoch 1 || Run Time:   63.5 | Load Time:    0.5 || F1:  74.34 | Prec:  60.65 | Rec:  96.02 || Ex/s:  12.18

===>  EVAL Epoch 1


0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


...

0% [█████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:48


Finished Epoch 10 || Run Time:   54.6 | Load Time:    0.5 || F1:  98.96 | Prec:  97.94 | Rec: 100.00 || Ex/s:  14.16

===>  EVAL Epoch 10


0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 10 || Run Time:    7.5 | Load Time:    0.2 || F1:  90.61 | Prec:  84.34 | Rec:  97.90 || Ex/s:  33.67

---------------------

Loading best model...
Training done.


tensor(92.2034)

In [18]:
# Evaluate the accuracy on the test data.
model.run_eval(test)

===>  EVAL Epoch 6


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 6 || Run Time:    6.5 | Load Time:    0.2 || F1:  87.20 | Prec:  87.20 | Rec:  87.20 || Ex/s:  39.25



tensor(87.2000)